In [1]:
import sqlite3

In [2]:
sql_db_path = './realtime/realtime_data/realtime.db'

In [3]:
table_ref_columns = {
    'power': ['time', 'generator'],
    'observation': ['station', 'obs_time'],
    'forecast': ['town', 'update_time', 'forecast_time']
}

In [4]:
def delete_duplicated_data(table_name, ref_columns):

    conn = sqlite3.connect(sql_db_path)
    cursor = conn.cursor()

    # 獲取表的列名
    cursor.execute(f'PRAGMA table_info({table_name})')
    columns = [column[1] for column in cursor.fetchall()]
    
    # 構建 蘭名 語句
    columns_str = ", ".join(columns)
    ref_str = ', '.join(ref_columns)

    # 創建輔助表
    cursor.execute(f'''
    CREATE TABLE IF NOT EXISTS temp_table AS 
    SELECT * FROM {table_name} WHERE 1=0
    ''')
    
    # 插入非重複記錄到輔助表中
    cursor.execute(f'''
    INSERT INTO temp_table ({columns_str})
    SELECT {columns_str} FROM {table_name}
    GROUP BY {ref_str}
    ''')
    
    # 刪除原表數據
    cursor.execute(f'DELETE FROM {table_name}')
    
    # 將輔助表數據插回原表
    cursor.execute(f'''
    INSERT INTO {table_name} ({columns_str})
    SELECT {columns_str} FROM temp_table
    ''')
    
    # 刪除輔助表
    cursor.execute('DROP TABLE temp_table')
    conn.commit()
    
    cursor.close()
    conn.close()

In [5]:
for table_name, ref_columns in table_ref_columns.items():
    delete_duplicated_data(table_name, ref_columns)